# Set up the project

In [1]:
import os

from label_studio_sdk import LabelStudio

LABEL_STUDIO_URL = os.environ.get('LABEL_STUDIO_URL')
API_KEY = os.environ.get('LABEL_STUDIO_API_KEY')

# for storages
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')

ls = LabelStudio(base_url=LABEL_STUDIO_URL, api_key=API_KEY)

# Start new project 
project = ls.projects.create(
    title='Storage Test Project',
    label_config='''
    <View>
      <Image name="image" value="$image" zoom="true"/>
      <PolygonLabels name="label" toName="image">
        <Label value="Airplane" background="red"/>
      </PolygonLabels>
    </View>
    '''
)

## Add a source (import) storage to the project

In [2]:
import_storage = ls.import_storage.s3.create(
    project=project.id,
    title="my storage", 
    bucket="htx-dev",
    prefix="dataset/training_set/dogs",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    use_blob_urls=True,  # "Treat every bucket object as a source file", more info: https://labelstud.io/guide/storage#Treat-every-bucket-object-as-a-source-file
    regex_filter=".*jpg"
)

In [3]:
import_storage

S3ImportStorage(aws_access_key_id=None, aws_secret_access_key=None, aws_session_token=None, aws_sse_kms_key_id=None, bucket='htx-dev', created_at=datetime.datetime(2025, 9, 30, 15, 7, 4, 603292, tzinfo=datetime.timezone.utc), description=None, id=5216, last_sync=None, last_sync_count=None, last_sync_job=None, meta={}, prefix='dataset/training_set/dogs', presign=True, presign_ttl=1, project=459758, recursive_scan=False, regex_filter='.*jpg', region_name=None, s3endpoint=None, status='initialized', synchronizable=True, title='my storage', traceback=None, type='s3', use_blob_urls=True)

## Sync import storage to get tasks from the storage 

In [14]:
sync_result = ls.import_storage.s3.sync(import_storage.id)

In [15]:
sync_result

S3ImportStorage(aws_access_key_id=None, aws_secret_access_key=None, aws_session_token=None, aws_sse_kms_key_id=None, bucket='htx-dev', created_at=datetime.datetime(2025, 9, 29, 21, 20, 56, 91931, tzinfo=datetime.timezone.utc), description=None, id=5180, last_sync=None, last_sync_count=None, last_sync_job='dfac54a3-db91-4e4c-9a45-80b32b855c81', meta={'attempts': 1, 'time_queued': '2025-09-29 21:21:09.438564+00:00'}, prefix='dataset/training_set/dogs', presign=True, presign_ttl=1, project=456308, recursive_scan=False, regex_filter='.*jpg', region_name=None, s3endpoint=None, status='queued', synchronizable=True, title='my storage', traceback=None, type='s3', use_blob_urls=True)

## Get status of synchronization for the storage

And wait until sync is completed. 

In [18]:
import time

while import_storage.status != 'completed':
    time.sleep(3.0)  # wait for 3 seconds    
    
    import_storage = ls.import_storage.s3.get(import_storage.id)
    print(import_storage.id, import_storage.title, import_storage.status)

print(f'Import storage {import_storage.id} is synced!')

Import storage 5180 is synced!


## Add a target (export) storage to the project

In [5]:
export_storage = ls.export_storage.s3.create(
    project=project.id,
    title="my export storage", 
    bucket="htx-dev",
    prefix="test_out",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    can_delete_objects=False  # if you delete annotations in Label Studio, the corresponding objects in the storage will not be deleted
)

In [6]:
export_storage

S3ExportStorage(aws_access_key_id=None, aws_secret_access_key=None, aws_session_token=None, aws_sse_kms_key_id=None, bucket='htx-dev', can_delete_objects=False, created_at=datetime.datetime(2025, 9, 30, 15, 7, 28, 766966, tzinfo=datetime.timezone.utc), description=None, id=3405, last_sync=None, last_sync_count=None, last_sync_job='7273680f-109f-4585-a7d4-9f6000a68712', meta={'attempts': 1, 'time_queued': '2025-09-30 15:07:28.793453+00:00'}, prefix='test_out', project=459758, regex_filter=None, region_name=None, s3endpoint=None, status='queued', synchronizable=True, title='my export storage', traceback=None, type='s3', use_blob_urls=False)

## Sync target storage to save annotations to the storage

Target storages sync automatically, so we don't need to call any methods to sync them. However, if you need to re-sync the storage from the scratch, you can call `sync` method.

In [7]:
sync_result = ls.export_storage.s3.sync(export_storage.id)
sync_result

S3ExportStorage(aws_access_key_id=None, aws_secret_access_key=None, aws_session_token=None, aws_sse_kms_key_id=None, bucket='htx-dev', can_delete_objects=False, created_at=datetime.datetime(2025, 9, 30, 15, 7, 28, 766966, tzinfo=datetime.timezone.utc), description=None, id=3405, last_sync=None, last_sync_count=None, last_sync_job='7273680f-109f-4585-a7d4-9f6000a68712', meta={'attempts': 1, 'time_queued': '2025-09-30 15:07:28.793453+00:00'}, prefix='test_out', project=459758, regex_filter=None, region_name=None, s3endpoint=None, status='queued', synchronizable=True, title='my export storage', traceback=None, type='s3', use_blob_urls=False)